In [1]:
from types import SimpleNamespace

# 假设config是你的配置字典
config = {
    'args': {
        'skip_connection': True, 
        'num_classes': 5, 
        'device': 'cuda', 
        'in_dim': 4096, 
        'out_dim': 4096, 
        'in_channels': 1, 
        'scale': 1
    }
}

args = SimpleNamespace(**config['args'])

# 现在你可以使用点符号来访问args的值
print(args.in_dim)  # 输出：4096

4096


In [1]:
import torch

print(torch.version.cuda)
print(torch.cuda.is_available())

12.1
True
